In [92]:
from binance import Client

In [93]:
import pandas as pd
import ta
import numpy as np
import time

In [94]:
client = Client()

In [95]:
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [96]:
df = getminutedata('ETHUSDT', '1m', '1000')

In [97]:
def applytechnicals(df):
    df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window=14, smooth_window=3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['rsi'] = ta.momentum.rsi(df.Close, window=14)
    df['macd'] = ta.trend.macd_diff(df.Close)
    df.dropna(inplace=True)

In [98]:
applytechnicals(df)

In [99]:
class Signals:
    
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
        
    def gettrigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
            dfx = dfx.append(mask, ignore_index=True)
        return dfx.sum(axis=0)
    
    def decide(self):
        self.df['trigger'] = np.where(self.gettrigger(), 1, 0)
        self.df['Buy'] = np.where((self.df.trigger) 
                            & (self.df['%K'].between(20,80)) 
                            & (self.df['%D'].between(20,80)) 
                            & (self.df.rsi > 50) 
                            & (self.df.macd > 0), 1, 0)
            
            

In [100]:
sig = Signals(df, 5)

In [101]:
sig.decide()

In [ ]:
def strategy(pair, qty, open_position=True):
    df = getminutedata(pair, '1m', '100')
    applytechnicals(df)
    sig = Signals(df, 25)
    sig.decide()
    print(f'current Close is '+ str(df.Close.iloc[-1]))
    
    if df.Buy.iloc[-1]:
        print('BUY ORDER')
        open_position = True
        
    while open_position:
        time.sleep(0.5)
        df = getminutedate(pair, '1m', '2')
        print(f'current Close' + str(df.Close.iloc[-1]))

